# 1. XGBoost

# 2. 실습

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve, auc

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, cross_val_predict
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy.stats
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

import time

# Data

- 데이터 출처
  - https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset/data

In [2]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data = data.drop(['id'], axis=1)
data = data[data['gender'] != 'Other']
data.head()

df = data.copy()
df.loc[:, ["hypertension", "heart_disease", "stroke"]] = data.loc[:, ["hypertension", "heart_disease", "stroke"]].applymap(lambda x: "Yes" if x == 1 else "No")
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,No,Yes,Yes,Private,Urban,228.69,36.6,formerly smoked,Yes
1,Female,61.0,No,No,Yes,Self-employed,Rural,202.21,NaN,never smoked,Yes
2,Male,80.0,No,Yes,Yes,Private,Rural,105.92,32.5,never smoked,Yes
3,Female,49.0,No,No,Yes,Private,Urban,171.23,34.4,smokes,Yes
4,Female,79.0,Yes,No,Yes,Self-employed,Rural,174.12,24.0,never smoked,Yes


## A. 전처리

In [3]:
X = df.drop(["stroke"], axis=1)
y = LabelEncoder().fit_transform(df['stroke'])

X_num = X.select_dtypes(include = 'number')
X_cat = X.select_dtypes(exclude = 'number')

X[X_num.columns] = SimpleImputer(strategy="mean").fit_transform(X_num)
X[X_cat.columns] = SimpleImputer(strategy="most_frequent").fit_transform(X_cat)

In [4]:
scaler = StandardScaler()

onehot = OneHotEncoder(drop = 'first', handle_unknown='ignore', sparse_output=False)

ct = ColumnTransformer([('scaler', scaler, X_num.columns),
                        ('onehot', onehot, X_cat.columns)], 
                       remainder='passthrough', n_jobs=-1)

ct

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 Index(['age', 'avg_glucose_level', 'bmi'], dtype='object')),
                                ('onehot',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 Index(['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'Residence_type', 'smoking_status'],
      dtype='object'))])

# 1. `XGBoost()`

## 파라미터

- **`learning_rate`**
  - 학습 단계별로 가중치를 얼마나 사용할 것인지 결정
  - 값이 작을수록 다음 단계의 결과물 적게 반영
  - 기본값 = 0.1, 일반적으로 **`0.01 ~ 0.2`** 결정
  - 범위 [0, 1]
- **`n_estimators`**
  - 생성할 weak learner의 수
  - 기본값 100, 일반적으로` `결정 
- **`min_child_weight`**
  - 관측치에 대한 인스턴스 가중치합의 최소
  - 값이 클수록 과적합 방지
  - 기본값 1
  - [0, ∞]
- **`min_split_loss (= gamma)`**
  - 리프노드의 추가 분할을 결정할 최소 손실 감소값
  - 기본값 = 0
  - 값이 클수록 과적합 방지
  - 범위: [0, ∞]
- **`max_depth`**
  - 트리 깊이
  - 기본값 6, 일반적으로 **`3 ~ 10`** 사용.
  - 값이 작을수록 과적합 방지
- **`subsample`**
  - 훈련 데이터의 일부를 무작위로 샘플링하는 비율
  - 기본값 1, 일반적으로 **`0.5 ~ 1`** 사이 값 사용
  - 각 부스팅 반복(iteration)마다 랜덤 샘플링 수행.
  - 값이 작을수록 과적합 방지
  - 범위: (0, 1]
- **`colsample_bytree`**
  - 각 트리마다 피쳐 샘플링 비율
  - 기본값 1, 일반적으로 **`0.5 ~ 1`** 사이 값 사용
- **`scale_pos_weight`**
  - 불균형 데이터셋의 균형 유지를 위해 사용
  - 기본값 1
- **`reg_lambda`**
  - L2규제(릿지) 가중치
  - 클수록 과적합 감소
  - 기본값 1
- **`reg_alpah`**
  - L1규제(라쏘) 가중치
  - 클수록 과적합 감소
  - 기본값 0

- `objective`
  - reg:linear : 회귀 (default)
  - binary:logistict : 이진분류
  - multi:softmax : 다중분류, 클래스 반환
  - multi:softprob : 다중분류, 확률 반환

- `eval_metric`
  - 목적함수에 따라 디폴트 값이 다름 (회귀 분석 : rmse, 클래스 분류: error)
  - rmse 
  - mae 
  - logloss
    - 예측 확률과 실제 라벨 사이의 차이
  - error
    - 예측라벨과 실제 라벨의 다른 비율
  - merror
  - mlogloss
  - auc